# System of linear equations

>    To solve $Ax=b$, where $A$ is an $n\times n$ matrix.

1. Pivoting
2. Timing between solving $Ax=b$ and $x = A^{-1}*b$
3. LU-factorization
4. Condition number
5. Jacobi method to solve a linear system


## Pivoting

Consider the linear system: 
$$
\left[\begin{matrix}\epsilon & 1 \\ 1 & 1\end{matrix}\right]
\left[\begin{matrix} x \\ y\end{matrix}\right] = 
\left[\begin{matrix} 1 \\ 2\end{matrix}\right]
$$
Without pivoting, the solution is given as
$$
y_1 = \frac{2-1/\epsilon}{1-1/\epsilon}, \quad x_1 = \frac{1-y}{\epsilon}.
$$
With pivoting, the solution is given as
$$
y_2 = \frac{1-2\epsilon}{1-\epsilon}, \quad x_2 = 2-y.
$$


In [1]:
import numpy as np

In [2]:
eps = 10**(-16)
y1 = (2.0-1.0/eps)/(1.0-1.0/eps)
x1 = (1.0-y1)/eps
y2 = (1.0-2.0*eps)/(1.0-eps)
x2 = 2.0-y2
print('epsilon equals to ', eps)
print('Without pivoting: ', x1, y1)
print('With    pivoting: ', x2, y2)

epsilon equals to  1e-16
Without pivoting:  2.220446049250313 0.9999999999999998
With    pivoting:  1.0 0.9999999999999999


### Quick conclusion
* Gaussian elimination without pivoting is *unstable*. 
* Gaussian elimination with pivoting is *stable*. 

## Timing

To have a feeling on the time requried between the following operations:
1. matrix-vector multiplication: $A\times b$
2. Solve a linear system: $Ax=b$
3. Solve a linear system by matrix inversion: $x=A^{-1}b$

*Caution:*

矩陣與向量乘法不能直接寫 `A*b`, 要用 `np.matmul(A,b)` 或是 `A.dot(b)`.

In [3]:
m = 2
A = m*np.identity(m) + np.random.random((m,m))
b = np.random.random((m,1))
print('A= ')
print(A)
print('b= ')
print(b)
print('A*b= ')
print(A*b)
print('np.matmul= ')
print(np.matmul(A,b))
print('.dot= ')
print(A.dot(b))

A= 
[[2.15274406 0.40673631]
 [0.1886845  2.94319257]]
b= 
[[0.3111491 ]
 [0.22915371]]
A*b= 
[[0.66982437 0.12655564]
 [0.04323775 0.6744435 ]]
np.matmul= 
[[0.7630295 ]
 [0.73315251]]
.dot= 
[[0.7630295 ]
 [0.73315251]]


In [4]:
from timeit import timeit

In [5]:
def multiplicationtime():
  return np.matmul(A, b)
def solvetime():
  return np.linalg.solve(A, b)
def solveinvtime():
  return np.linalg.inv(A)*b

Generate a $m\times m$ non-singular random matrix $A$ and a $m\times 1$ random vector $b$.

In [6]:
m = 2000
A = m*np.identity(m) + np.random.random((m,m))
b = np.random.random((m,1))

In [7]:
print('m=   ', m)
print('The time takes for A times b is ', timeit(stmt=multiplicationtime, number=10))
print('The time takes for solving Ax=b is ', timeit(stmt=solvetime, number=10))
print('The time takes for A^(-1) times b is ', timeit(stmt=solveinvtime, number=10))

m=    2000
The time takes for A times b is  0.042201954999995905
The time takes for solving Ax=b is  2.455234055999995
The time takes for A^(-1) times b is  10.980635658999972


### Quick conclusion
The solution to the linear system is $x = A^{-1}b$, but in practice we should not use this formula to evaluate the solution.

## LU-factorization

In [8]:
import numpy as np
import scipy
import scipy.linalg

In [9]:
A = np.array([
    [1.0, -2.0, 3.0, 0.0],
    [3.0, -6.0, 9.0, 3.0],
    [2.0, 1.0, 4.0, 1.0],
    [1.0, -2.0, 2.0, 2.0]
])
print(A)

[[ 1. -2.  3.  0.]
 [ 3. -6.  9.  3.]
 [ 2.  1.  4.  1.]
 [ 1. -2.  2.  2.]]


In [10]:
### LU-factorization of A
P, L, U = scipy.linalg.lu(A)
print(f'P = ')
print(P)
print(f'L = ')
print(L)
print(f'U = ')
print(U)
print(f'P*L*U= ')
print(P.dot(L).dot(U))

P = 
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
L = 
[[ 1.          0.          0.          0.        ]
 [ 0.66666667  1.          0.          0.        ]
 [ 0.33333333  0.          1.          0.        ]
 [ 0.33333333  0.         -0.          1.        ]]
U = 
[[ 3. -6.  9.  3.]
 [ 0.  5. -2. -1.]
 [ 0.  0. -1.  1.]
 [ 0.  0.  0. -1.]]
P*L*U= 
[[ 1. -2.  3.  0.]
 [ 3. -6.  9.  3.]
 [ 2.  1.  4.  1.]
 [ 1. -2.  2.  2.]]


## Condition number

In [11]:
import numpy as np
from numpy import linalg as LA

In [12]:
A = np.array([
    [1.01, -2.0, 3.0, 0.0],
    [3.0, -6.0, 9.0, 3.0],
    [2.0, 1.0, 4.0, 1.0],
    [1.0, -2.0, 2.0, 2.0]
])
print('A= ')
print(A)
print('condition number of A in 2-norm = ', LA.cond(A))

A= 
[[ 1.01 -2.    3.    0.  ]
 [ 3.   -6.    9.    3.  ]
 [ 2.    1.    4.    1.  ]
 [ 1.   -2.    2.    2.  ]]
condition number of A in 2-norm =  55.51183485384228


In [13]:
B = np.array([
    [1.01, 0.99],
    [0.99, 1.01]
])
print('B= ')
print(B)
print('condition number of B in inf-norm = ', LA.cond(B, np.inf))

B= 
[[1.01 0.99]
 [0.99 1.01]]
condition number of B in inf-norm =  99.99999999999991


In [14]:
from timeit import timeit

In [15]:
def conditiontime():
  return LA.cond(A)

In [16]:
m = 100
A = np.random.random((m,m))

In [17]:
print('The time takes to evaluate a matrix of size ', m, ' is ', timeit(stmt=conditiontime, number=10))

The time takes to evaluate a matrix of size  100  is  0.026157093000051646


## Jacobi method to solve a linear system

In [18]:
import numpy as np
from numpy import linalg as LA

Construct a $m\times m$ diagonal dominant matrix by adding a diagonal matrix with its element to be $m$. 

In [19]:
m = 2000
# A: the m-by-m diagonal dominant matrix
A = m*np.identity(m) + np.random.random((m,m))
# xe: the exact solution
xe = np.random.random((m,1))
# b: the right hand side vector, b = Ax
b = A.dot(xe)
print('shape of A = ', np.shape(A))
print('shape of b = ', np.shape(b))

shape of A =  (2000, 2000)
shape of b =  (2000, 1)


Calculate the condition number of A

In [20]:
print('condition number of A in 2-norm = ', LA.cond(A))

condition number of A in 2-norm =  1.5137468501953022


Split the matrix into $D$, $L$ and $U$.

Define `dAinv` to be $D^{-1}$.

In [21]:
### Splitting A into D, L and U
D = np.diag(A)
dA = D.reshape(m,1)
D = np.diag(D)
U = np.triu(A, 1)
L = A - D - U
dAinv = np.reciprocal(dA)

Jacobi iteration:
$$
x^{(k+1)} = D^{-1}(b - (L+U)x^{(k)}).
$$
* `mtrJ`: $L+U$
* `xk`: $x^{(k)}$
* `xkp1`: $x^{(k+1)}$

We measure the residual $\|b - Ax\|$ and the difference between two steps $\|x^{(k+1)}-x^{(k)}\|$. 

In [22]:
### Jacobi iteration
mtrJ = L + U
k = 0
xk = b
tolr = 1e-15
itmx = 100

b_inf = np.linalg.norm(b, np.inf)
rel_res_inf = 1.0
rel_dif_inf = 1.0

print('')
print('Jacobi iteration:')
print('')

while ( (rel_res_inf>tolr) and (rel_dif_inf>tolr) and (k<itmx) ):
    # Jacobi iterative step
    xkp1 = dAinv*(b - mtrJ.dot(xk))

    # relative residual
    res = b - np.matmul(A, xkp1)
    rel_res_inf = np.linalg.norm(res, np.inf)/b_inf
    dif = xkp1 - xk
    rel_dif_inf = np.linalg.norm(dif, np.inf)/np.linalg.norm(xk, np.inf)
    k += 1
    print('Iter %4d, relative residual: %.4e, relative difference: %.4e' % (k, rel_res_inf, rel_dif_inf) )
    xk = xkp1


Jacobi iteration:

Iter    1, relative residual: 9.5120e+02, relative difference: 1.3151e+00
Iter    2, relative residual: 4.7868e+02, relative difference: 1.4945e+00
Iter    3, relative residual: 2.3913e+02, relative difference: 1.4972e+00
Iter    4, relative residual: 1.1946e+02, relative difference: 1.5043e+00
Iter    5, relative residual: 5.9677e+01, relative difference: 1.4902e+00
Iter    6, relative residual: 2.9812e+01, relative difference: 1.5188e+00
Iter    7, relative residual: 1.4893e+01, relative difference: 1.4580e+00
Iter    8, relative residual: 7.4400e+00, relative difference: 1.5327e+00
Iter    9, relative residual: 3.7168e+00, relative difference: 1.3066e+00
Iter   10, relative residual: 1.8567e+00, relative difference: 1.5378e+00
Iter   11, relative residual: 9.2756e-01, relative difference: 9.2271e-01
Iter   12, relative residual: 4.6337e-01, relative difference: 1.5474e+00
Iter   13, relative residual: 2.3148e-01, relative difference: 4.2379e-01
Iter   14, relativ

Here we solve the system $Ax=b$ directly by `np.linalg.solve`. Denote this solution as `xs`.

In [23]:
### Solve linear system directly
xs = np.linalg.solve(A, b)
print('relative residual: ', np.linalg.norm(b - A.dot(xs), np.inf)/b_inf)

relative residual:  3.771328343821646e-15


The error between the solutions obtained using two approaches.

In [24]:
# the error of the solution obtained using the Jacobi method
err_jac = np.linalg.norm(xk-xe, np.inf)
# the error of the solution obtained using the linear solver
err_lin = np.linalg.norm(xs-xe, np.inf)
print('Error of Jacobi = ', err_jac)
print('Error of linalg.solve = ', err_lin)

Error of Jacobi =  2.1094237467877974e-15
Error of linalg.solve =  4.773959005888173e-15
